<a href="https://colab.research.google.com/github/bill7845/colab_repo/blob/master/dacon_%EC%A0%9C%EC%A3%BC%EC%B9%B4%EB%93%9C%EC%82%AC%EC%9A%A9%EB%9F%89%EC%98%88%EC%B8%A1/NOTE_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# < DACON - 제주 빅데이터 경진대회(카드 사용금액 예측) >
---

<br>

## Day 2 정리
  * Q&A 읽고 정리

<br>

* Q. 이용 고객수(CSTMR_CNT)가 이용횟수(CNT)보다 많은 경우?
  * A
    * 카드 승인 후 승인 취소 고객이 존재
    * ex) A,B,C가 각각 1만원,2만원,3만원 결제 후 C가 승인 취소한다면 이용자 수는 3명 이용건수는 2건, 이용 금액은 1만원+2만원=3만원으로 집계됨.

<br>

* Q. 예측 시점?
  * A
    * 1. 1차 공개되는 데이터는 2019.01~2020.03까지의 데이터임
    * 2. Public Score 기간 동안은 1에서 주어진 데이터로 2020.04월 데이터를 예측해야함. (평가지표 RMSLE, 제주지역에 3배 가중치 부여)
    * 3. Public Score 기간 종료 후 2020.04 데이터를 추가로 공개
    * 4. Final Score는 2019.01~2020.04의 데이터를 바탕으로 2020.07월을 예측해야함.

<br>

* Q. 거래 내역이 없는 경우? (ex. 강원지역의 그외 기타 분류안된 오락관련 서비스업의 경우 19년 5월에 2건이 전부임)
  * A. 데이터가 나타나지 않는 경우 크게 2가지 경우임.
    * 1. 기간 내 카드 소비가 발생하지 않는 경우.
    * 2. 이용자 수가 적어 비식별화 조치로 데이터가 삭제된 경우. (대부분 여기에 해당)
  * 비식별화 조치로 삭제된 데이터 금액은 전체 사용액 대비 상당히 낮은 비율임.
  * 모든 기간에서(2019.01~2020.03) 집계에 빠져있는 업종이라도 사용량이 많다면 이후 데이터(2020.04, 2020.07)에서 집계될 수 있음

<br>


* 평가 지표(RMSLE)에 관한 정보
  * RMSLE 특성상 큰 금액을 비슷하게 예측하는 것보다 작은 금액을 큰 차이 없이 예측하는것이 더 나은 전략일 수 있음.
  * RMSLE 특성상 실제값이 0이거나 예측값이 0인데 조금이라도 예측값,실제값과 다르면 큰 페널티를 받음 따라서, 지난 달 결제금액이 0원이였던 지역/업종의 예측값을 매뉴얼하게 0으로 바꿀것을 추천. (or 1로)

<br>

* lagging에 대해..
  * Public Score기간에는 lagging을 1달~3달 등 자유롭게 사용가능. but Final LeaderBord기간의 7월 금액 예측에는 앞서 사용하였던 모델을 그대로 적용불가(2020.05,2020.06데이터 없음) 

<br>

* submission score
  * dacon base line : 7.09526 , 188위 ( 2020.06.22 )

In [4]:
## import libarary

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor


import warnings
warnings.filterwarnings("ignore")

import seaborn as sns
sns.set()

In [5]:
import matplotlib as mpl
import matplotlib.pyplot as plt
 
%config InlineBackend.figure_format = 'retina'
 
!apt -qq -y install fonts-nanum
 
import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager._rebuild()

fonts-nanum is already the newest version (20170925-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 33 not upgraded.


In [7]:
data = pd.read_csv('/content/drive/My Drive/Colab code/dacon_카드사용액 예측/data/201901-202003.csv') # data load

In [ ]:
## data preprocessing

data = data.fillna('') # 결측값 ''로 대체

## 날짜 처리
def grap_year(data): # year 
    data = str(data)
    return int(data[:4])

def grap_month(data): # month 
    data = str(data)
    return int(data[4:])

data['year'] = data['REG_YYMM'].apply(lambda x: grap_year(x))
data['month'] = data['REG_YYMM'].apply(lambda x: grap_month(x))
data = data.drop(['REG_YYMM'], axis=1) # 기존 날짜컬럼 삭제

In [ ]:
data.head(5)

,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT,year,month
0,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,4,311200,4,2019,1
1,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2,7,1374500,8,2019,1
2,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2,6,818700,6,2019,1
3,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,4,1717000,5,2019,1
4,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,4,3,1047300,3,2019,1


In [ ]:
## data preprocessing

df = data.copy()
df = df.drop(['CARD_CCG_NM', 'HOM_CCG_NM'], axis=1) # 카드이용지역_시군구 (가맹점 주소 기준), 거주지역_시군구 (고객 집주소 기준) 삭제

columns = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'year', 'month']
df = df.groupby(columns).sum().reset_index(drop=False) # 지역, 업종 별 월간 총 사용 금액을 예측하기 위한 groupby

In [ ]:
df.head()

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,year,month,CSTMR_CNT,AMT,CNT
0,강원,건강보조식품 소매업,강원,20s,1,1,2019,1,4,311200,4
1,강원,건강보조식품 소매업,강원,20s,1,1,2019,2,3,605000,3
2,강원,건강보조식품 소매업,강원,20s,1,1,2019,6,3,139000,3
3,강원,건강보조식품 소매업,강원,20s,1,1,2019,8,3,27500,3
4,강원,건강보조식품 소매업,강원,20s,1,1,2019,9,3,395500,3


In [ ]:
# unique values

print(df['year'].unique())
print(df['month'].unique())
print(df['AGE'].unique())
print(df['CARD_SIDO_NM'].unique())
print(df['HOM_SIDO_NM'].unique())
print(df['STD_CLSS_NM'].unique())

[2019 2020]
[ 1  2  6  8  9 11  3  4  5  7 10 12]
['20s' '30s' '40s' '50s' '60s' '70s' '10s']
['강원' '경기' '경남' '경북' '광주' '대구' '대전' '부산' '서울' '세종' '울산' '인천' '전남' '전북'
 '제주' '충남' '충북']
['강원' '경기' '경남' '경북' '광주' '대구' '부산' '서울' '인천' '전북' '충남' '충북' '대전' '세종'
 '울산' '전남' '제주']
['건강보조식품 소매업' '골프장 운영업' '과실 및 채소 소매업' '관광 민예품 및 선물용품 소매업'
 '그외 기타 분류안된 오락관련 서비스업' '그외 기타 스포츠시설 운영업' '그외 기타 종합 소매업' '기타 대형 종합 소매업'
 '기타 수상오락 서비스업' '기타 외국식 음식점업' '기타 주점업' '기타음식료품위주종합소매업' '내항 여객 운송업' '마사지업'
 '면세점' '버스 운송업' '비알콜 음료점업' '빵 및 과자류 소매업' '서양식 음식점업' '수산물 소매업' '슈퍼마켓'
 '스포츠 및 레크레이션 용품 임대업' '여관업' '여행사업' '욕탕업' '육류 소매업' '일반유흥 주점업' '일식 음식점업'
 '자동차 임대업' '전시 및 행사 대행업' '중식 음식점업' '차량용 가스 충전업' '차량용 주유소 운영업' '체인화 편의점'
 '택시 운송업' '피자 햄버거 샌드위치 및 유사 음식점업' '한식 음식점업' '호텔업' '화장품 및 방향제 소매업'
 '휴양콘도 운영업' '정기 항공 운송업']


In [ ]:
df = df[((df['month']==4) | (df['month']==7)) ]
df

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,year,month,CSTMR_CNT,AMT,CNT
10,강원,건강보조식품 소매업,강원,20s,2,1,2019,4,4,266000,4
13,강원,건강보조식품 소매업,강원,20s,2,1,2019,7,4,30300,4
22,강원,건강보조식품 소매업,강원,20s,2,2,2019,4,3,250000,4
25,강원,건강보조식품 소매업,강원,20s,2,2,2019,7,3,266000,4
32,강원,건강보조식품 소매업,강원,30s,1,2,2019,4,30,1868100,36
...,...,...,...,...,...,...,...,...,...,...,...
1057350,충북,휴양콘도 운영업,충북,60s,1,5,2019,7,43,2606700,56
1057362,충북,휴양콘도 운영업,충북,60s,2,5,2019,4,24,1910500,42
1057365,충북,휴양콘도 운영업,충북,60s,2,5,2019,7,18,926300,31
1057377,충북,휴양콘도 운영업,충북,70s,1,5,2019,4,11,555600,16


In [ ]:
# 인코딩
dtypes = df.dtypes # 각 컬럼별 dtype
encoders = {}
for column in df.columns:
    if str(dtypes[column]) == 'object': # dtype이 object인 컬럼이면  ## CARD_SIDO_NM, STD_CLSS_NM, HOM_SIDO_NM, AGE
        # LabelEncoder로 encoding
        encoder = LabelEncoder()
        encoder.fit(df[column])
        encoders[column] = encoder

# encoding된 데이터로 변경        
df_num = df.copy()        
for column in encoders.keys():
    encoder = encoders[column]
    df_num[column] = encoder.transform(df[column])

In [ ]:
df_num

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,year,month,CSTMR_CNT,AMT,CNT
10,0,0,0,1,2,1,2019,4,4,266000,4
13,0,0,0,1,2,1,2019,7,4,30300,4
22,0,0,0,1,2,2,2019,4,3,250000,4
25,0,0,0,1,2,2,2019,7,3,266000,4
32,0,0,0,2,1,2,2019,4,30,1868100,36
...,...,...,...,...,...,...,...,...,...,...,...
1057350,16,40,16,5,1,5,2019,7,43,2606700,56
1057362,16,40,16,5,2,5,2019,4,24,1910500,42
1057365,16,40,16,5,2,5,2019,7,18,926300,31
1057377,16,40,16,6,1,5,2019,4,11,555600,16


In [ ]:
# feature, target 설정
train_num = df_num.sample(frac=1, random_state=0) ## df.sample()을 통해 sampling 단, frac=1이므로 이 경우 전체 데이터를 셔플링한 효과
train_features = train_num.drop(['CSTMR_CNT', 'AMT', 'CNT'], axis=1)
train_target = np.log1p(train_num['AMT'])

In [ ]:
# 훈련
model = RandomForestRegressor(n_jobs=-1, random_state=0)
model.fit(train_features, train_target)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=-1, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)

In [ ]:
# 예측 템플릿 만들기
CARD_SIDO_NMs = df_num['CARD_SIDO_NM'].unique()
STD_CLSS_NMs  = df_num['STD_CLSS_NM'].unique()
HOM_SIDO_NMs  = df_num['HOM_SIDO_NM'].unique()
AGEs          = df_num['AGE'].unique()
SEX_CTGO_CDs  = df_num['SEX_CTGO_CD'].unique()
FLCs          = df_num['FLC'].unique()
years         = [2020]
months        = [4, 7]

temp = []
for CARD_SIDO_NM in CARD_SIDO_NMs:
    for STD_CLSS_NM in STD_CLSS_NMs:
        for HOM_SIDO_NM in HOM_SIDO_NMs:
            for AGE in AGEs:
                for SEX_CTGO_CD in SEX_CTGO_CDs:
                    for FLC in FLCs:
                        for year in years:
                            for month in months:
                                temp.append([CARD_SIDO_NM, STD_CLSS_NM, HOM_SIDO_NM, AGE, SEX_CTGO_CD, FLC, year, month])
temp = np.array(temp)
temp = pd.DataFrame(data=temp, columns=train_features.columns)

In [ ]:
temp.head()

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,year,month
0,0,0,0,1,2,1,2020,4
1,0,0,0,1,2,1,2020,7
2,0,0,0,1,2,2,2020,4
3,0,0,0,1,2,2,2020,7
4,0,0,0,1,2,3,2020,4


In [ ]:
# 예측
pred = model.predict(temp)
pred = np.expm1(pred)
temp['AMT'] = np.round(pred, 0)
temp['REG_YYMM'] = temp['year']*100 + temp['month']
temp = temp[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
temp = temp.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False)

In [ ]:
# 디코딩 
temp['CARD_SIDO_NM'] = encoders['CARD_SIDO_NM'].inverse_transform(temp['CARD_SIDO_NM'])
temp['STD_CLSS_NM'] = encoders['STD_CLSS_NM'].inverse_transform(temp['STD_CLSS_NM'])

In [ ]:
# 제출 파일 만들기
submission = pd.read_csv('/content/drive/My Drive/Colab code/dacon_카드사용액 예측/data/submission.csv', index_col=0)
submission = submission.drop(['AMT'], axis=1)
submission = submission.merge(temp, left_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
submission.index.name = 'id'
submission.to_csv('/content/drive/My Drive/Colab code/dacon_카드사용액 예측/submission/NOTE_1.csv', encoding='utf-8-sig')
submission

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,5.429987e+08
1,202004,강원,골프장 운영업,6.744380e+09
2,202004,강원,과실 및 채소 소매업,1.779664e+09
3,202004,강원,관광 민예품 및 선물용품 소매업,1.719409e+08
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,1.709555e+08
...,...,...,...,...
1389,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,1.815036e+09
1390,202007,충북,한식 음식점업,2.376249e+10
1391,202007,충북,호텔업,5.692680e+08
